In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/health-and-lifestyle-dataset/health_activity_data.csv


In [16]:
health = pd.read_csv("/kaggle/input/health-and-lifestyle-dataset/health_activity_data.csv")
health.head()

,ID,Age,Gender,Height_cm,Weight_kg,BMI,Daily_Steps,Calories_Intake,Hours_of_Sleep,Heart_Rate,Blood_Pressure,Exercise_Hours_per_Week,Smoker,Alcohol_Consumption_per_Week,Diabetic,Heart_Disease
0,1,56,Male,164,81,30.72,5134,1796,8.6,102,137/72,8.1,No,7,No,No
1,2,69,Male,156,82,20.86,12803,1650,4.5,103,129/65,3.7,No,7,No,No
2,3,46,Female,158,65,30.93,16408,1756,4.3,74,127/68,3.2,Yes,0,No,No
3,4,32,Male,197,87,31.19,18420,2359,4.1,116,125/86,8.5,No,5,No,No
4,5,60,Male,157,63,29.37,17351,2556,5.1,111,100/64,8.5,Yes,8,No,No


In [17]:
health.shape
health.dtypes

ID                                int64
Age                               int64
Gender                           object
Height_cm                         int64
Weight_kg                         int64
BMI                             float64
Daily_Steps                       int64
Calories_Intake                   int64
Hours_of_Sleep                  float64
Heart_Rate                        int64
Blood_Pressure                   object
Exercise_Hours_per_Week         float64
Smoker                           object
Alcohol_Consumption_per_Week      int64
Diabetic                         object
Heart_Disease                    object
dtype: object

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# ==========================================
# 1. DATA PREPARATION & MODEL TRAINING
# ==========================================
df = health.copy()
df[['Systolic', 'Diastolic']] = df['Blood_Pressure'].str.split('/', expand=True).astype(int)

le_gender = LabelEncoder()
df['Gender_Encoded'] = le_gender.fit_transform(df['Gender'])

binary_map = {'No': 0, 'Yes': 1}
for col in ['Smoker', 'Diabetic', 'Heart_Disease']:
    df[col] = df[col].map(binary_map)

def define_lifestyle(row):
    score = 0
    if row['Daily_Steps'] > 10000: score += 1
    if row['Smoker'] == 0: score += 1
    if 18.5 <= row['BMI'] <= 24.9: score += 1
    if row['Exercise_Hours_per_Week'] >= 3: score += 1
    return 'Optimal' if score >= 3 else 'Balanced' if score == 2 else 'At Risk'

df['Lifestyle_Status'] = df.apply(define_lifestyle, axis=1)

features = ['Age', 'Gender_Encoded', 'Height_cm', 'Weight_kg', 'BMI', 
            'Daily_Steps', 'Calories_Intake', 'Hours_of_Sleep', 'Heart_Rate', 
            'Systolic', 'Diastolic', 'Exercise_Hours_per_Week', 'Smoker', 
            'Alcohol_Consumption_per_Week', 'Diabetic', 'Heart_Disease']

X = df[features]
y = df['Lifestyle_Status']

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

print("✅ Model trained and crash-proofed!")

# ==========================================
# 2. INTERACTIVE USER INPUT
# ==========================================

print("\n--- Health Dashboard Input ---")
age = int(input("Age: "))
gender = input("Gender (Male/Female): ").strip().capitalize()
height = int(input("Height (cm): "))
weight = int(input("Weight (kg): "))

# Auto-calculate BMI if user is unsure (Weight / Height^2)
bmi_calc = round(weight / ((height/100)**2), 2)
print(f"Calculated BMI: {bmi_calc}")

steps = int(input("Daily Steps: "))
calories = int(input("Calories Intake: "))
sleep = float(input("Hours of Sleep: "))
hr = int(input("Heart Rate: "))
bp_input = input("Blood Pressure (Format 120/80): ")

# CRASH FIX: Safe split for Blood Pressure
try:
    sys, dia = map(int, bp_input.split('/'))
except:
    print("⚠️ Invalid BP format. Using default 120/80.")
    sys, dia = 120, 80

exercise = float(input("Exercise Hours per Week: "))
smoker = input("Smoker (Yes/No): ").strip().capitalize()
alcohol = int(input("Alcohol (Drinks per Week): "))
diabetic = input("Diabetic (Yes/No): ").strip().capitalize()
heart_disease = input("Heart Disease (Yes/No): ").strip().capitalize()

# ==========================================
# 3. PREDICTION
# ==========================================

gen_enc = le_gender.transform([gender])[0]
user_vector = [
    age, gen_enc, height, weight, bmi_calc, steps, calories, sleep, hr,
    sys, dia, exercise, binary_map[smoker], alcohol, 
    binary_map[diabetic], binary_map[heart_disease]
]

prediction = model.predict([user_vector])[0]

print("\n" + "⭐" * 20)
print(f"LIFESTYLE STATUS: {prediction.upper()}")
print("⭐" * 20)

if prediction == 'At Risk':
    print("Suggestion: Your activity or vitals show room for improvement. Sending data to Gemini for a custom plan...")

✅ Model trained and crash-proofed!

--- Health Dashboard Input ---


Age:  24
Gender (Male/Female):  Female
Height (cm):  155
Weight (kg):  55


Calculated BMI: 22.89


Daily Steps:  1200
Calories Intake:  5000
Hours of Sleep:  4
Heart Rate:  52
Blood Pressure (Format 120/80):  120/80
Exercise Hours per Week:  8
Smoker (Yes/No):  No
Alcohol (Drinks per Week):  45
Diabetic (Yes/No):  No
Heart Disease (Yes/No):  No



⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐
LIFESTYLE STATUS: OPTIMAL
⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [24]:
!pip install google-genai

In [ ]:
# Import the NEW library correctly
try:
    from google import genai
except ImportError:
    print("Library not found. Please run !pip install -U google-genai and RESTART KERNEL.")

from kaggle_secrets import UserSecretsClient

# 1. SETUP THE CLIENT
try:
    user_secrets = UserSecretsClient()
    gemini_key = user_secrets.get_secret("GEMINI_API_KEY")
except:
    # Fallback if not using Kaggle Secrets
    gemini_key = "YOUR_GEMINI_API_KEY_HERE"

# Initialize the new client
client = genai.Client(api_key=gemini_key)

def get_gemini_guidance(user_data, status):
    """
    Uses the new google-genai SDK for lifestyle suggestions.
    """
    
    prompt = f"""
    You are a professional Health & Lifestyle Coach.
    ML Model Prediction for this user: {status.upper()}
    
    User Statistics:
    - Age/Gender: {user_data['age']} / {user_data['gender']}
    - Activity: {user_data['steps']} steps/day, {user_data['exercise']} hrs/week
    - Nutrition: {user_data['calories']} kcal, Sleep: {user_data['sleep']}h
    - Vitals: BP {user_data['bp']}, HR {user_data['hr']}, BMI {user_data.get('bmi', 'N/A')}
    - Habits: Smoker: {user_data['smoker']}, Alcohol: {user_data['alcohol']} units/week
    
    Task:
    1. Explain why our ML model categorized them as '{status}'.
    2. Provide 3 high-impact, actionable lifestyle changes.
    3. Keep it professional, concise, and encouraging.
    """

    # New SDK syntax: models.generate
    response = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt
    )
    
    return response.text

# 2. EXECUTE
print("\n" + "="*30)
print("✨ GENERATING AI HEALTH INSIGHTS...")
print("="*30 + "\n")

try:
    # Using the 'user_profile' and 'prediction' variables from your previous blocks
    ai_guide = get_gemini_guidance(user_profile, prediction)
    print(ai_guide)
except Exception as e:
    print(f"An error occurred: {e}")


✨ GENERATING AI HEALTH INSIGHTS...

Okay, let's break down this assessment.

**Explanation of "Optimal" Prediction:**

Our model likely categorizes you as "Optimal" primarily because of your healthy blood pressure (120/80) and heart rate (52), coupled with a healthy BMI (22.89). Despite some lifestyle factors that could be improved, these key vital signs indicate a strong baseline of health at your age.

**Actionable Lifestyle Changes (Prioritized):**

While your overall profile is positive, focusing on these three areas will yield the greatest benefits:

1.  **Prioritize Sleep:** Aim for 7-9 hours of quality sleep per night. Sleep deprivation impacts everything from mood and energy levels to metabolism and immune function. Experiment with a consistent sleep schedule and a relaxing bedtime routine. This is the most impactful change you can make.

2.  **Reduce Alcohol Consumption:** 45 units per week is significantly above recommended guidelines. Gradually reduce your intake to align w